## Get Candlestick Data 

In [1]:
import requests
import calendar
import dateutil.parser as parser
from dateutil.relativedelta import relativedelta
from datetime import datetime, timezone
import yaml
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [2]:
now = datetime.now()

In [3]:
def convert_date(utc_time): 
    parsed_date = parser.parse(utc_time)
    var_date=parsed_date.date()
    var_time=parsed_date.time()
    var_f_time=var_time.hour
    var_julian_date=parsed_date.timetuple().tm_yday
    var_weekday=parsed_date.weekday()
    var_weekday_name=calendar.day_name[parsed_date.weekday()]
    return var_date, var_time, var_f_time, var_julian_date, var_weekday, var_weekday_name

In [4]:
with open ('back_test_pipeline_settings.yaml') as ymlfile:
    cfg = yaml.safe_load(ymlfile)
    instrument = cfg['currency']['instrument']

In [5]:
print('Instrument:', instrument)

Instrument: USD_CAD


### The Configs for Run:

In [6]:
with open ('config.yml') as ymlfile:
    cfg = yaml.safe_load(ymlfile)
    oanda_api_key = cfg['creds']['oanda_api']
    account_number = cfg['creds']['account_number'] 

### Select the Currency Pair

In [7]:
Load_10K_Records=True
Load_20K_Records=True

#  'EUR_USD','USD_CAD','EUR_GBP','EUR_AUD','EUR_CHF',
#  'GBP_USD','GBP_CHF','GBP_NZD','GBP_AUD','GBP_CAD',
#  'AUD_CAD','AUD_CHF','AUD_NZD','NZD_USD','EUR_CAD',
#  'USD_CHF','CAD_CHF','NZD_CAD','AUD_USD','EUR_NZD',
#  'NZD_CHF',

currency_pairs = [instrument]


timeframe = "H4"
#D #H1 #H4 M30
# https://developer.oanda.com/rest-live-v20/instrument-df/#CandlestickGranularity
price_char = "M"
#M(midpoint candles) #B(bid candles) #A(ask candles) #BA
price_com = "mid"
#mid #bid #ask

# def of OANDA request variable
provider_api_url = 'https://api-fxpractice.oanda.com/v3/accounts/{}/orders'.format(account_number)
request_headers = {
    "Authorization": oanda_api_key,
    "Accept-Datetime-Format": "RFC3339",
    "Connection": "Keep-Alive",
    "Content-Type": "application/json;charset=UTF-8"
}


In [8]:
provider_authorization = 'Bearer {0}'.format(oanda_api_key)

headers = {
    'Content-Type': 'application/json',
    'Authorization': provider_authorization,
}

# Get Candlesticks Data

In [9]:
params_count = (
    ('price', price_char),
    ('count', '5000'),
    ('granularity', timeframe),
)


In [10]:
for pair in currency_pairs:
    first_response = requests.get('https://api-fxpractice.oanda.com/v3/instruments/{}/candles'.format(pair), 
                            headers=headers,
                            params=params_count).json()

In [11]:
if Load_10K_Records:
    
    #### Load 10K data
        
    datetime_object = parser.parse(first_response['candles'][0]['time'])
    date= datetime_object - relativedelta(years=3)  
    from_date = date.replace(tzinfo=timezone.utc).timestamp()
    params_date = (
        ('count', '5000'),
        ('price', price_char),
        ('from', from_date),
        ('granularity', timeframe),)

    second_response = requests.get('https://api-fxpractice.oanda.com/v3/instruments/{}/candles'.format(pair),
                                   headers=headers,
                                   params=params_date).json()
    datetime_object_15K = parser.parse(second_response['candles'][0]['time'])
    first_response= first_response['candles']  
    second_response= second_response['candles']
    second_response.extend(first_response)
    
    if Load_20K_Records:

        #### Load 15K data

        date= datetime_object_15K - relativedelta(years=3)  
        from_date = date.replace(tzinfo=timezone.utc).timestamp()
        params_date = (
            ('count', '5000'),
            ('price', price_char),
            ('from', from_date),
            ('granularity', timeframe),)

        third_response = requests.get('https://api-fxpractice.oanda.com/v3/instruments/{}/candles'.format(pair),
                                       headers=headers,
                                       params=params_date).json()
        datetime_object_20K = parser.parse(third_response['candles'][0]['time'])
        third_response= third_response['candles']
        third_response.extend(second_response)

        #### Load 20K data

        date= datetime_object_20K - relativedelta(years=3)  
        from_date = date.replace(tzinfo=timezone.utc).timestamp()
        params_date = (
            ('count', '5000'),
            ('price', price_char),
            ('from', from_date),
            ('granularity', timeframe),)

        fourth_response = requests.get('https://api-fxpractice.oanda.com/v3/instruments/{}/candles'.format(pair),
                                       headers=headers,
                                       params=params_date).json()
        datetime_object_20K = parser.parse(fourth_response['candles'][0]['time'])
        fourth_response= fourth_response['candles']
        fourth_response.extend(third_response)

        response=fourth_response
    else:
       response=second_response 
else:
    response=first_response['candles']

In [12]:
filename = "{}_{}.csv".format(pair, timeframe)
output = []
all_candlesticks = response

for i in range (len(all_candlesticks)):
    result= (convert_date(response[i]['time']))
    output.append([(result[0]),(result[1]),(result[2]),(result[3]),(result[4]),(result[5]),
                    response[i]['time'],
                    response[i]['volume'], 
                    response[i][price_com]['o'],
                    response[i][price_com]['h'],
                    response[i][price_com]['l'],
                    response[i][price_com]['c']])
    
output = pd.DataFrame(output)
output.columns = ['Date','Time','f_time','julian_date','Weekday','Weekday_Name','UTC_Time', 'Volume', 'Open', 'High', 'Low', 'Close']
data = output.to_csv(filename, header = True, index = False)
data = pd.read_csv(filename)

In [13]:
data = data.drop_duplicates()
data = data.to_csv(filename, header = True, index = False)
data = pd.read_csv(filename)

In [14]:
data.shape

(19342, 12)

In [15]:
data.head(5)

,Date,Time,f_time,julian_date,Weekday,Weekday_Name,UTC_Time,Volume,Open,High,Low,Close
0,2009-11-11,22:00:00,22,315,2,Wednesday,2009-11-11T22:00:00.000000000Z,1332,1.04473,1.04549,1.04240,1.04350
1,2009-11-12,02:00:00,2,316,3,Thursday,2009-11-12T02:00:00.000000000Z,1265,1.04346,1.04454,1.04182,1.04332
2,2009-11-12,06:00:00,6,316,3,Thursday,2009-11-12T06:00:00.000000000Z,4160,1.04334,1.04926,1.04291,1.04886
3,2009-11-12,10:00:00,10,316,3,Thursday,2009-11-12T10:00:00.000000000Z,6177,1.04888,1.05468,1.04694,1.05054
4,2009-11-12,14:00:00,14,316,3,Thursday,2009-11-12T14:00:00.000000000Z,11578,1.05058,1.05382,1.04796,1.05182


In [16]:
data.tail(5)

,Date,Time,f_time,julian_date,Weekday,Weekday_Name,UTC_Time,Volume,Open,High,Low,Close
19337,2022-01-21,02:00:00,2,21,4,Friday,2022-01-21T02:00:00.000000000Z,4789,1.25224,1.25369,1.25140,1.25180
19338,2022-01-21,06:00:00,6,21,4,Friday,2022-01-21T06:00:00.000000000Z,5179,1.25182,1.25348,1.25090,1.25252
19339,2022-01-21,10:00:00,10,21,4,Friday,2022-01-21T10:00:00.000000000Z,4821,1.25250,1.25326,1.25061,1.25176
19340,2022-01-21,14:00:00,14,21,4,Friday,2022-01-21T14:00:00.000000000Z,12638,1.25173,1.25562,1.25146,1.25534
19341,2022-01-21,18:00:00,18,21,4,Friday,2022-01-21T18:00:00.000000000Z,6098,1.25536,1.25878,1.25446,1.25827


## Simple Moving Average (SMA)

In [17]:
data['SMA_5'] = data['Close'].rolling(window=5).mean().round(4)
data['SMA_10'] = data['Close'].rolling(window=10).mean().round(4)
data['SMA_20'] = data['Close'].rolling(window=20).mean().round(4)

## Simple Moving Average Range

In [18]:
data['F_SMA_5'] = data['Close'] - data['SMA_5']
data['F_SMA_10'] = data['Close'] - data['SMA_10']
data['F_SMA_20'] = data['Close'] - data['SMA_20']

In [19]:
data = data.drop_duplicates()
data = data.to_csv(filename, header = True, index = False)
data = pd.read_csv(filename)

In [20]:
data.tail()

,Date,Time,f_time,julian_date,Weekday,Weekday_Name,UTC_Time,Volume,Open,High,Low,Close,SMA_5,SMA_10,SMA_20,F_SMA_5,F_SMA_10,F_SMA_20
19337,2022-01-21,02:00:00,2,21,4,Friday,2022-01-21T02:00:00.000000000Z,4789,1.25224,1.25369,1.25140,1.25180,1.2497,1.2498,1.2502,0.00210,0.00200,0.00160
19338,2022-01-21,06:00:00,6,21,4,Friday,2022-01-21T06:00:00.000000000Z,5179,1.25182,1.25348,1.25090,1.25252,1.2506,1.2501,1.2504,0.00192,0.00242,0.00212
19339,2022-01-21,10:00:00,10,21,4,Friday,2022-01-21T10:00:00.000000000Z,4821,1.25250,1.25326,1.25061,1.25176,1.2517,1.2502,1.2504,0.00006,0.00156,0.00136
19340,2022-01-21,14:00:00,14,21,4,Friday,2022-01-21T14:00:00.000000000Z,12638,1.25173,1.25562,1.25146,1.25534,1.2527,1.2506,1.2506,0.00264,0.00474,0.00474
19341,2022-01-21,18:00:00,18,21,4,Friday,2022-01-21T18:00:00.000000000Z,6098,1.25536,1.25878,1.25446,1.25827,1.2539,1.2515,1.2509,0.00437,0.00677,0.00737


## Price Range

In [21]:
data['O-H'] = data['Open'] - data['High']
data['O-L'] = data['Open'] - data['Low']
data['O-C'] = data['Open'] - data['Close']
data['H-L'] = data['High'] - data['Low']
data['H-C'] = data['High'] - data['Close']
data['L-C'] = data['Low'] - data['Close']

data['Direction'] = data['O-C'].apply(lambda x: 1 if x<0 else 0)

data['col_1'] = data['Open'] - data['Close']

for value in data['col_1']:   
    if value > 0:
        data['col_2'] = data['High'] - data['Open']
        data['col_3'] = data['Close'] - data['Low']
    else:
        data['col_2'] = data['High'] - data['Close']
        data['col_3'] = data['Open'] - data['Low']

#Two Previous Candlesticks 
data['col_4'] = data['col_1'].shift(1)
data['col_5'] = data['col_1'].shift(2)

In [22]:
data = data.dropna()
data = data.to_csv(filename, header = True, index = False)
data = pd.read_csv(filename)

In [23]:
data.tail()

,Date,Time,f_time,julian_date,Weekday,Weekday_Name,UTC_Time,Volume,Open,High,...,O-C,H-L,H-C,L-C,Direction,col_1,col_2,col_3,col_4,col_5
19318,2022-01-21,02:00:00,2,21,4,Friday,2022-01-21T02:00:00.000000000Z,4789,1.25224,1.25369,...,0.00044,0.00229,0.00189,-0.00040,0,0.00044,0.00189,0.00084,-0.00168,-0.00418
19319,2022-01-21,06:00:00,6,21,4,Friday,2022-01-21T06:00:00.000000000Z,5179,1.25182,1.25348,...,-0.00070,0.00258,0.00096,-0.00162,1,-0.00070,0.00096,0.00092,0.00044,-0.00168
19320,2022-01-21,10:00:00,10,21,4,Friday,2022-01-21T10:00:00.000000000Z,4821,1.25250,1.25326,...,0.00074,0.00265,0.00150,-0.00115,0,0.00074,0.00150,0.00189,-0.00070,0.00044
19321,2022-01-21,14:00:00,14,21,4,Friday,2022-01-21T14:00:00.000000000Z,12638,1.25173,1.25562,...,-0.00361,0.00416,0.00028,-0.00388,1,-0.00361,0.00028,0.00027,0.00074,-0.00070
19322,2022-01-21,18:00:00,18,21,4,Friday,2022-01-21T18:00:00.000000000Z,6098,1.25536,1.25878,...,-0.00291,0.00432,0.00051,-0.00381,1,-0.00291,0.00051,0.00090,-0.00361,0.00074


In [24]:
data.shape

(19323, 30)